In [2]:
from pytorch_pretrained_bert import BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
tokenizer = BertTokenizer.from_pretrained('../../../bert-base-chinese/vocab_unk.txt', do_lower_case=True)

In [9]:
def get_positions(data_list, map_str):
    map_str = tokenizer.tokenize(map_str)
    map_str = [i.replace('#', '') for i in map_str]
    map_str = ''.join(map_str)
    # 如果只由一个词组成
    for word in data_list:
        if map_str.lower() in word.lower():
            start_id = end_id = data_list.index(word)
            return start_id, end_id

    start_id = -1
    end_id = -1
    for idx, word in enumerate(data_list):
        if start_id != - 1 and end_id != -1:
            return start_id, end_id
        if map_str.startswith(word):
            start_id = end_id = idx
            while end_id+1 < len(data_list) and data_list[end_id+1] in map_str:
                if "".join(data_list[start_id:end_id+2]) == map_str:
                    print("".join(data_list[start_id:end_id+2]))
                    return start_id, end_id+1
                end_id += 1
            find_str = ""
            for idx in range(start_id, end_id+1):
                find_str = find_str + data_list[idx]
            if find_str != map_str:
                pre_extend = (data_list[start_id-1] if start_id > 0 else "") + find_str
                last_extend = find_str + (data_list[end_id+1] if end_id < len(data_list)-1 else "")
                pre_last_extend = (data_list[start_id-1] if start_id > 0 else "")+ find_str + (data_list[end_id+1] if end_id < len(data_list)-1 else "")
                print("need:{} \nfind:{} \n pre:{}\n last:{} \n pre_las:{}".format(map_str,find_str, pre_extend,pre_extend,pre_last_extend))
                if map_str in pre_extend:
                    start_id -= 1
                elif map_str in last_extend:
                    end_id += 1
                elif map_str in pre_last_extend:
                    start_id -= 1
                    end_id += 1
                else:
                    start_id = -1
                    end_id = -1
    if start_id != -1 and end_id != -1:
        return start_id, end_id
    for idx, word in enumerate(data_list[:-1]):
        if map_str in (word+data_list[idx+1]):
            return idx, idx+1
    # print("word_list{}  map_str {} loss".format(data_list, map_str))
    return start_id, end_id

In [10]:
text = '《李烈钧自述》是2011年11月1日人民日报出版社出版的图书，作者是李烈钧'
word_list = tokenizer.tokenize(text)

In [11]:
get_positions(word_list, '人民日报出版社')

人民日报出版社


(14, 20)

In [33]:
word_list[10:17]

['机', '械', '工', '业', '出', '版', '社']